In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from os import path, environ
from datetime import datetime
from services.zayev.environment.market_simulator import MarketSimulator
from services.zayev.service_layer.forester import Forester
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import accuracy_score
from matplotlib.dates import relativedelta
from apps.environment.service_layer.data_transformer import DataTransformer
import pandas as pd
from joblib import load
import pathlib
import pytz

2024-01-01 22:33:57.635567: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
chosen_stock = 11
(current_file_path, new_file_path) = DataTransformer.get_new_file_path(chosen_stock=chosen_stock)

rf = load(pathlib.Path(current_file_path))

nwb_starting_time_step = datetime(year=2023, month=11, day=10)
# now_time = pytz.utc.localize(datetime(year=2024, month=1, day=1))
now_time = pytz.utc.localize(datetime.now())

nwb_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nwb_env_config = {
    "db_params": nwb_db_params, 
    "max_episode_steps": 1000, 
    "the_current_time_step": nwb_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": False,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True,
    "training_stock_index": chosen_stock
} 
nwb_dformer = DataTransformer(env_config=nwb_env_config)
nwb_dfs = nwb_dformer.create_input_dataframes()
nwb_market = MarketSimulator(env_config=nwb_env_config, data_frames=nwb_dfs)
nwb_market.reset()

total_init_potfolio = nwb_market.wallet_state
for initd in range(100):
    data_initd = initd+1
    total_init_potfolio = total_init_potfolio + nwb_market.stock_data[data_initd]["price_snapshot"]*nwb_market.shares_data[data_initd]
print(total_init_potfolio)
print(nwb_market.the_current_time_step - relativedelta(hours=100))

nwb_forest = Forester(market=nwb_market)
total_potfolio = nwb_market.wallet_state
latest_action = None
for ind in range(100):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]

initial_total_potfolio = total_potfolio
print(total_potfolio)
print(nwb_market.the_current_time_step, now_time)
for indx in range(200):
    if indx < 14:
        acts = np.full(100, 15)
        rand_acts = nwb_forest.act_random(acts)
        nwb_market.step(rand_acts)
    else:
        rand_acts = nwb_forest.market.get_mov_avg_actions()
        # print(rand_acts)
        acts_list = []
        iny = 1
        # rand_acts = np.full(100, 15)
        rand_acts = nwb_forest.act_random(rand_acts, use_best_stks=False)
        for an_act in rand_acts:
            acts_list.append((iny, an_act))
            iny=iny+1
        print(nwb_market.the_current_time_step, nwb_market.the_current_time_step.day, now_time.day)
        print(acts_list)
        if nwb_market.the_current_time_step >= now_time :
            latest_action = acts
            break
        nwb_market.step(rand_acts)
        print("")

print(nwb_market.wallet_state)

/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:392: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='ffill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:393: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stcks_concatenated_df[field] = stcks_concatenated_df.groupby('index')[field].fillna(method='bfill')
/Users/mirbilal/Desktop/MobCommission/commissionV2/apps/environment/service_layer/data_transformer.py:422: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cmmdts_concatenated_df[field] = cmmdts_concatenated_df.groupby('index')[fi

268284.31
2023-11-05 20:00:00+00:00
268284.31
2023-11-10 00:00:00+00:00 2024-01-01 22:34:03.035321+00:00
[ 4.25579119  1.48635161  8.22005669  1.9619437   0.17695647  0.23261207
  3.9299381  11.31741551  0.87608773  7.2268908   4.04379552  5.92773401
  4.1671491   3.91868802  2.21617231  5.60217106  3.7909682   2.67828762
  0.80857916  9.02912958  0.04303911  5.07803708  4.81427673 20.
  3.30601782  3.20657237  0.         11.90962588  7.12689925  3.00862199
  5.30825064  1.74478741  2.09848046  4.32092991  2.08293568  7.24509871
  3.30913769  9.41378377  7.61113194  4.8872509   9.19837479  3.27990172
  6.88379679  0.52820945  3.1059329   4.3008495   7.89414613  1.76119262
  2.6196648   5.42127037  0.83825836  0.86689307  1.90934531  1.37373176
  4.05079833  2.16524178  5.75821414  8.28872369  4.12497472 13.82812762
  4.77229227  3.54365082  6.8382262   9.09956613  9.04352453  1.66842725
  7.20089413  8.08065922  9.82743176  2.96374678  4.0585631   4.58938919
 10.46946194  5.61186088  2

KeyError: 1

In [3]:
total_potfolio = nwb_market.wallet_state
for ind in range(100):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]
print(total_potfolio)
print(nwb_market.the_current_time_step - relativedelta(hours=100))
final_total_potfolio = total_potfolio
gain = final_total_potfolio - initial_total_potfolio
print("gain", gain)

KeyError: 1